In [1]:
from nufeb_tools import utils,plot
from scipy.integrate import odeint
import numpy as np
import cv2
import pandas as pd
from itertools import combinations
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
import time
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from numba import jit
from pathlib import Path
import tables

ImportError: DLL load failed while importing hdf5extension: The specified procedure could not be found.

In [2]:
from dask import delayed
from dask.distributed import Client
import dask.array as da
import dask.dataframe as dd
#client = Client(n_workers=4)



In [3]:
x.h5.keys()

NameError: name 'x' is not defined

In [6]:
f = str((Path.home()) / '.nufeb_tools' / 'data' / 'Run_60_18_63_1_2021-06-23/trajectory.h5')

In [7]:
#xd= da.from_array(x.h5, chunks='auto')
ddfs = dd.read_hdf(f,'/*')

#ddf = dd.concat(ddfs)

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [4]:
#x = utils.get_data(directory= r'D:\runs\Run_33_66_72_1_2021-06-24')

x = utils.get_data(test=True)

# Assign mother cells

In [3]:
df = x.positions
df['mother_cell'] = df[df.Timestep==0].ID#.astype(pd.Int64Dtype())
df

,Timestep,ID,type,radius,x,y,z,mother_cell
0,0,75,2,4.465000e-07,0.000020,0.000023,9.400000e-07,75.0
1,0,50,1,8.200000e-07,0.000013,0.000016,2.140000e-06,50.0
2,0,19,1,8.400000e-07,0.000005,0.000008,2.680000e-06,19.0
3,0,59,1,8.950000e-07,0.000041,0.000016,3.310000e-06,59.0
4,0,42,1,7.200000e-07,0.000017,0.000006,6.170000e-06,42.0
...,...,...,...,...,...,...,...,...
43624,10000,297,2,4.548962e-07,0.000093,0.000084,9.540719e-06,NaN
43625,10000,456,2,4.462032e-07,0.000095,0.000085,9.558846e-06,NaN
43626,10000,202,2,4.359640e-07,0.000093,0.000085,9.569131e-06,NaN
43627,10000,987,2,4.884362e-07,0.000093,0.000086,9.507520e-06,NaN


In [176]:
#@jit(parallel=True)
def get_mothers(df,dataset):
    for time in df.Timestep.unique():
        for cell in df[df.Timestep==time].ID:
            if df[df.ID==cell].head(1).mother_cell.isna().values[0]:
                cell_type = df[df.ID==cell].type.unique()[0]
                dist = dataset.get_neighbor_distance(cell,time)
                dist_type = dist[dist.type==cell_type]
                neighborID = dist_type[dist_type.Distance == dist_type.Distance.min()].ID.values[0]
                mother = df[df.ID==neighborID].head(1).mother_cell.values[0]
                mask = df[df.ID==cell]
                df.loc[mask.index,'mother_cell'] = mother
    df.mother_cell = df.mother_cell.astype('Int64')
    return df
df2 = delayed(get_mothers) (df,x)

In [178]:
df2.compute()

TypeError: h5py objects cannot be pickled

In [164]:

def plot_colony():
    timepoint = 10000
    scale = 1
    tp = df[df.Timestep == timepoint]
    img_size = 1024
    bk = 255 * np.ones(shape=[img_size, img_size, 3], dtype=np.uint8)

    N = 100
    red = np.ones((N, 3))
    red[:, 0] = 255
    red[:, 1] = np.linspace(0, 255,N,dtype=int)
    red[:, 2] = np.linspace(65, 255,N,dtype=int)

    green = np.ones((N, 3))
    green[:, 0] = np.linspace(65, 255,N,dtype=int)
    green[:, 2] = np.linspace(0, 255,N,dtype=int)
    green[:, 1] = 255

    for i, colony in enumerate(tp.mother_cell.unique()):
        colony_type = tp[tp.mother_cell==colony].type
        colors = tuple(np.random.randint(0,256, 3).astype('int'))
        if not colony_type.empty:
            #if colony_type.unique()[0] == 1:

                #colors = green
            #elif colony_type.unique()[0] == 2:
            #print(colony)
                #colors = red
            for cell in tp[tp.mother_cell==colony].itertuples():
                #print(cell)
                
                xloc = round(cell[5]/x.metadata['Dimensions'][0]*img_size)
                yloc = round(cell[6]/x.metadata['Dimensions'][1]*img_size)
                radius = round(cell[4]/x.metadata['Dimensions'][0]*img_size*scale)
                
                cv2.circle(bk,center = (xloc,yloc),radius = radius,color = (int(colors[0]),int(colors[1]),int(colors[2])),thickness = -1)
    f, ax = plt.subplots(figsize=(15,15))
    ax.imshow(bk)
    ax.set_axis_off()


In [ ]:
plot_colony()